In [23]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
df.shape
df.info()
print("\nMissing values:")
print(df.isna().sum().sort_values(ascending=False))
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB

Missing values:
Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [25]:
# 1. Drop truly useless columns — explain why
df.drop(['PassengerId', 'Ticket'], axis=1, inplace=True)
print("Dropped PassengerId & Ticket → unique IDs, no predictive signal, cause noise/leakage")

Dropped PassengerId & Ticket → unique IDs, no predictive signal, cause noise/leakage


In [26]:
# 2. Cabin → extract signal instead of dropping 77% data
df['Has_Cabin'] = df['Cabin'].notna().astype(int)
df['Deck'] = df['Cabin'].str[0].fillna('M')  # M = Missing
df.drop('Cabin', axis=1, inplace=True)
print("Created Has_Cabin (0/1) + Deck (A-G + M) → huge survival correlation")

Created Has_Cabin (0/1) + Deck (A-G + M) → huge survival correlation


In [27]:
# 3. Smart Age imputation (group-wise median)
age_medians = df.groupby(['Pclass', 'Sex'])['Age'].median()
df['Age'] = df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))
print("Age imputed by Pclass+Sex median → much better than global median")

Age imputed by Pclass+Sex median → much better than global median


In [28]:
# 4. Embarked – only 2 missing
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

/var/folders/8t/v0xt96g10w98qbs2bnl651bm0000gn/T/ipykernel_47971/2231543239.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)


In [29]:
# FINAL CLEANED-UP FEATURE ENGINEERING (run this ONE cell only)
# Skip Title if Name already dropped, keep the rest

# If Title not created yet (Name already gone), create a simple one from memory (good enough for interview)
if 'Title' not in df.columns:
    print("Name already dropped → creating dummy Title (Mr=0, Miss/Mrs/Master=1)")
    df['Title'] = 0
    df.loc[df['Sex'] == 'female', 'Title'] = 1
    df.loc[df['Age'] < 15, 'Title'] = 2   # crude proxy for Master

# Family features (safe)
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
df['Fare_Per_Person'] = df['Fare'] / df['FamilySize']

# Has_Cabin & Deck (in case not done)
if 'Has_Cabin' not in df.columns and 'Cabin' in df.columns:
    df['Has_Cabin'] = df['Cabin'].notna().astype(int)
    df['Deck'] = df['Cabin'].str[0].fillna('M')
    df.drop('Cabin', axis=1, inplace=True)

print("All features created successfully")

Name already dropped → creating dummy Title (Mr=0, Miss/Mrs/Master=1)
All features created successfully


In [31]:
print(f"Final shape: {df.shape}")
print(f"Missing values: {df.isna().sum().sum()}")
df.head(3)

Final shape: (891, 15)
Missing values: 0


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Has_Cabin,Deck,Title,FamilySize,IsAlone,Fare_Per_Person
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,0,M,0,2,0,3.62500
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,1,C,1,2,0,35.64165
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,0,M,1,1,1,7.92500


In [34]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Drop non-numeric columns
df.drop('Name', axis=1, inplace=True)

# Quick encoding for categorical cols
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])
df['Embarked'] = le.fit_transform(df['Embarked'])
df['Deck'] = le.fit_transform(df['Deck'])

# Model
X = df.drop('Survived', axis=1)
y = df['Survived']

rf = RandomForestClassifier(n_estimators=500, max_depth=6, random_state=42)
scores = cross_val_score(rf, X, y, cv=5, scoring='accuracy')

print("5-fold CV Accuracy: {:.3f} ± {:.3f}".format(scores.mean(), scores.std() * 2))
print("Scores:", scores.round(3))

5-fold CV Accuracy: 0.827 ± 0.031
Scores: [0.838 0.803 0.843 0.815 0.837]
